In [1]:
#from julia.api import Julia
#jl = Julia(compiled_modules=False)
#from julia import Main

from numba import jit
import numpy as np
from scipy.optimize import newton
import matplotlib.pyplot as plt
%matplotlib inline

#julia_calc_E = Main.include('calc_E.jl')

In [ ]:
# https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19950021346.pdf

In [2]:
# export PATH="/Applications/Julia-1.4.app/Contents/Resources/julia/bin:$PATH"

In [13]:
N = int(1e7)
e = np.random.rand(N)
M = np.random.rand(N) * 2*np.pi

In [14]:
#E_julia = julia_calc_E(e, M)[0]

In [21]:
def calc_E(e, M):
    M[M > np.pi] -= 2*np.pi
    α = (3 * np.pi**2 + 1.6 * (np.pi**2 - np.pi * abs(M))/(1 + e))/(np.pi**2 - 6)
    d = 3 * (1 - e) + α * e
    q = 2 * α * d * (1 - e) - M**2
    r = 3 * α * d * (d - 1 + e) * M + M**3
    w = (abs(r) + np.sqrt(q**3 + r**2))**(2/3)
    E1 = (2 * r * w / (w**2 + w*q + q**2) + M)/d
    f2 = e * np.sin(E1)
    f3 = e * np.cos(E1)
    f0 = E1 - f2 - M
    f1 = 1 - f3
    δ3 = -f0 / (f1 - f0 * f2 / (2 * f1))
    δ4 = -f0 / (f1 + f2 * δ3 / 2 + δ3**2 * f3/6)
    δ5 = -f0 / (f1 + δ4*f2/2 + δ4**2*f3/6 - δ4**3*f2/24)
    
    E = E1 + δ5
    E[E < 0] += 2 * np.pi
    return E

In [22]:
%%timeit
E_py = calc_E(e, M)

2.85 s ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
